In [1]:
import pandas as pd
import json
import os # Operaciones relacionadas con el sistema de archivos
import datetime
import re

ETL preliminar en local para la comprensión de los datos.

### Metadatos Google

In [2]:
# Función para leer y filtrar los archivos JSON
def filtrar_archivos_json(directorio, categoria_busqueda):
    ''' Esta función, llamada filtrar_archivos_json, se encarga de buscar y filtrar 
        archivos JSON en un directorio específico en función de una categoría de búsqueda 
        proporcionada.   '''
    resultados = []
    for archivo in os.listdir(directorio):
            ruta_archivo = os.path.join(directorio, archivo)

            # Leer el contenido del archivo JSON
            with open(ruta_archivo) as f:
                archivo_json = f.readlines()

            # Cargar archivo JSON linea por linea
            for linea in archivo_json:
                data = json.loads(linea)
                # Verificar si la categoría buscada está presente en el archivo JSON
                categorias = data.get("category", [])
                if isinstance(categorias, list) and any(categoria_busqueda.lower() in categoria.lower() for categoria in categorias):
                    resultados.append(data)
    return resultados

In [4]:
# Filtrar los archivos JSON para la categoría "restaurant"
restaurant = filtrar_archivos_json('../PROYECTO_FINAL_EQUIPO/Google/metadata-sitios', "restaurant")

In [5]:
# Convertir los resultados a un pandas DataFrame
df_restaurant = pd.DataFrame(restaurant)

In [7]:
#Se tienen 181724 establecimientos que son restaurantes.
df_restaurant.shape

(181724, 15)

In [8]:
#1061 restaurantes duplicados
df_restaurant['gmap_id'].duplicated().sum()

1061

In [18]:
#Revisión de restaurantes duplicados
restaurantes_duplicados = df_restaurant[df_restaurant['gmap_id'].duplicated(keep=False)]
# Ordena los restaurantes duplicados por gmap_id
restaurantes_duplicados_ordenados = restaurantes_duplicados.sort_values(by='gmap_id')
restaurantes_duplicados_ordenados.head(2)


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
63908,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...
64969,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...


In [9]:
#Paso 1: Eliminar duplicados
df_restaurant.drop_duplicates(subset='gmap_id',inplace=True) 

In [10]:
#Tenemos 180,663 establecimientos con 15 atributos, de los cuales es necesario cuales tienen
#realmente reseñas para los futuros análisis.
df_restaurant.shape

(180663, 15)

### Revisión de algunas columnas a eliminar

In [154]:
# relative_results hace referencia de recomendación a otros negocios
# pero dado que nos interesa hacer un sistema propio de recomendaciones, lo dejaremos de lado.
df_restaurant['relative_results'][1] 

['0x5495b2cea4d3a7f7:0xd788b88074f97be6',
 '0x5495a8ba9e39a85f:0xfbe5938763a14568',
 '0x54944d01d969e923:0xf8cc1e24f8aaeb50',
 '0x54944d27a2b501fd:0xde3ce099feaa6d78',
 '0x5495a9b18b24d4d7:0x93026337fd559ca3']

In [155]:
#Muchos de los datos que 'relative_results' arroja no están en nuestro DF
df_restaurant[df_restaurant['gmap_id']== '0x5495b2cea4d3a7f7:0xd788b88074f97be6']

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url


In [156]:
#Nos dice el estado en el que se encuentra el restaurante al momento que se 
# extrajeron los datos por lo que no es relevante para nuestro análisis
df_restaurant['state'][1]

'Permanently closed'

### Reviews-estados

Se hace la carga de las Reviews para obtener las que pertenezcan a los restaurantes y eliminar los restaurantes que no tengan Reviews. 

In [ ]:
ruta_reviews_estados = "../PROYECTO_FINAL_EQUIPO/Google/reviews-estados/"
rutas_subfolders = [f.path for f in os.scandir(ruta_reviews_estados) if f.is_dir()]

In [ ]:
def df_json(ruta_folder):
    '''Unión de archivos en un DataFrame '''
# Inicializa una lista para almacenar los objetos JSON de los archivos
    data = []
    
    # Itera a través de los archivos en la carpeta especificada
    for file_name in os.listdir(ruta_folder):
        # Construye la ruta completa de cada archivo
        file_path = os.path.join(ruta_folder, file_name)
        # Abre el archivo en modo lectura ('r')
        with open(file_path, 'r') as file:
            # Itera a través de las líneas del archivo
            for line in file:
                # Elimina espacios en blanco al principio y al final de la línea
                line = line.strip()
                # Intenta cargar la línea como un objeto JSON
                json_obj = json.loads(line)
                    # Agrega el objeto JSON a la lista de datos
                data.append(json_obj)

    # Crea un DataFrame de pandas a partir de la lista de datos
    df = pd.DataFrame(data)
    
    # Devuelve el DataFrame resultante
    return df

In [ ]:
def df_json_general(subfolders):
    # Inicializa una lista para almacenar DataFrames individuales
    df_list = []
    
    # Itera a través de las subcarpetas especificadas
    for subfolder in subfolders:
        # Llama a la función read_json_files para procesar los archivos JSON en la subcarpeta
        df = df_json(subfolder)
        
        # Agrega el DataFrame resultante a la lista
        df_list.append(df)
    
    # Devuelve una lista de DataFrames, uno por cada subcarpeta
    return df_list

In [ ]:
lista_df = df_json_general(rutas_subfolders)
dfreviews = pd.concat(lista_df, ignore_index=True)

In [35]:
#80624057 rows × 8 columns
dfreviews.shape

(80624057, 8)

In [ ]:
#Transformación del dato tiempo.
def convertir_a_datetime(timestamp_ms):
    fecha_hora = datetime.datetime.fromtimestamp(timestamp_ms / 1000.0)  # Dividir por 1000 para convertir a segundos
    return fecha_hora

In [36]:
dfreviews['time'] = dfreviews['time'].apply(convertir_a_datetime)


In [39]:
dfreviews['time'] = pd.to_datetime(dfreviews['time'], format='%Y-%m-%d', errors='coerce')

#### SELECCIÓN DE RESTAURANTES CON RESEÑAS

In [22]:
#Obtenemos los establecimientos que tienen Reviews
id_establecimientos_con_reviews = dfreviews["gmap_id"].unique()

In [23]:
#Se tienen 1,178,143 establecimientos únicos o que tienen una review.
id_establecimientos_con_reviews.shape[0]

1178143

In [25]:
#Filtrar los restaurantes que tienen Reviews.
df_restaurantes = df_restaurant[df_restaurant["gmap_id"].isin(id_establecimientos_con_reviews)].reset_index(drop=True)

In [47]:
#Finalmente tenemos 84,326 restaurantes con reseñas. 
df_restaurantes.shape

(84326, 15)

#### Filtrar reseñas de Restaurantes

In [27]:
#Restaurantes únicos
filtro=df_restaurantes['gmap_id'].unique()

In [30]:
#Reseñas de restaurantes únicos
dfreviews = dfreviews[dfreviews["gmap_id"].isin(filtro)]

In [31]:
#Obtenemos 4,709,086 de reseñas para los #84,326 restaurantes
dfreviews.shape

(4709086, 8)

#### Pequeña limpieza de la columna texto para su posterior uso. 

In [55]:
dfreviews['text']=dfreviews['text'].fillna('')

In [57]:
def limpiar_texto(texto):
    if isinstance(texto, str):  # Verifica si el valor es una cadena de texto
        # Utiliza una expresión regular para eliminar caracteres especiales y números
        texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto)
        return texto_limpio
    else:
        return texto  # Devuelve el valor original si no es una cadena de texto

In [58]:
dfreviews['text'] = dfreviews['text'].apply(limpiar_texto)

In [65]:
dfreviews[dfreviews['user_id']== '102762323164773988805']

,user_id,name,time,rating,text,business_id
1692040,102762323164773988805,Chris Petetsen,2021-03-01,5,Very friendly people,0x88530dbbd6d510f7:0xbf3bfe826a5897ca


In [59]:
#Aún es necesario tener criterios más estrictos para la selección de datos 
# a analizar.
dfreviews.to_csv('Reviews_Datawarahouse.csv',index=False)